In [6]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [7]:
# Creating your first graph and running it in a session
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

# create a sessioin, initializes the variables, and evaluates, and close the session
# sess = tf.Session() # tf2 doesn't have Session()
sess = tf.compat.v1.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [8]:
# better way
with tf.compat.v1.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [9]:
# global variable initilizer
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [10]:
# interactive session in jupyter-notebook
sess = tf.compat.v1.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [20]:
# Linear regression with TensorFlow
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

# theta using normal equation
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

with tf.compat.v1.Session() as sess:
    theta_value = theta.eval()
    
print('theta_normal = \n', theta_value)

theta_normal = 
 [[-3.7158638e+01]
 [ 4.3620414e-01]
 [ 9.3944781e-03]
 [-1.0677545e-01]
 [ 6.4319843e-01]
 [-4.0679406e-06]
 [-3.7808842e-03]
 [-4.2343181e-01]
 [-4.3694979e-01]]


In [35]:
# implementing gradient descent

# normalize the input feature vector
from sklearn.preprocessing import normalize

scaled_housing_data = normalize(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# manually computing the gradients
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.compat.v1.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# manual optimizer
# gradients = 2/m * tf.matmul(tf.transpose(X), error) # analytical
# gradients = tf.gradients(mse, [theta])[0] # using reverse-mode autodiff
# training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)

# built-in optimizer
# optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate) # Gradient Descent Optimizer
optimizer = tf.compat.v1.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # Gradient Descent Optimizer
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 5.591224
Epoch 100 MSE = 1.3438287
Epoch 200 MSE = 1.3341441
Epoch 300 MSE = 1.329798
Epoch 400 MSE = 1.3277627
Epoch 500 MSE = 1.3267397
Epoch 600 MSE = 1.3261603
Epoch 700 MSE = 1.3257759
Epoch 800 MSE = 1.3254774
Epoch 900 MSE = 1.3252171
[[ 0.88026917]
 [ 0.6611962 ]
 [ 1.1976064 ]
 [-0.70200884]
 [-0.5577024 ]
 [ 1.1239457 ]
 [ 0.04214306]
 [-0.48006928]
 [-0.52962536]]


In [42]:
# Mini-batch Gradient Descent
A = tf.compat.v1.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.compat.v1.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]
